In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
df=pd.read_excel('netflix_db.xlsx',index_col=None)
df

,show_title,cast_member_1,cast_member_2,cast_member_3,genre_1,genre_2,genre_3,mood_1,mood_2
0,Black Mirror,Jesse Plemons,Cristin Milioti,Jimmi Simpson,TV Dramas,British,Sci-Fi TV,Mind-Bending,Ominous
1,Savage Beauty,Rosemary Zimu,Dumisani Mbebe,Nthati Moshesh,TV Dramas,South African,Social Issue TV Dramas,Emotional,NaN
2,Shameless (U.S.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Our Great National Parks,Barack Obama,NaN,NaN,Family Watch Together TV,Nature &amp; Ecology Documentaries,Science &amp; Nature TV,Inspiring,NaN
4,How to Become a Mob Boss,Peter Dinklage,NaN,NaN,Docuseries,US TV Shows,NaN,Dark,NaN
...,...,...,...,...,...,...,...,...,...
279,The 100,Eliza Taylor,Paige Turco,Bob Morley,Political TV Shows,TV Dramas,Sci-Fi TV,Violent,NaN
280,The Lincoln Lawyer,Manuel Garcia-Rulfo,Neve Campbell,Becki Newton,TV Dramas,Courtroom TV Shows,TV Shows Based on Books,Witty,Suspenseful
281,Crime Scene: The Times Square Killer,NaN,NaN,NaN,Crime TV Shows,Docuseries,US TV Shows,Investigative,NaN
282,Formula 1: Drive to Survive,NaN,NaN,NaN,British,Sports &amp; Fitness,Docuseries,Exciting,NaN


In [3]:
df_null=df.isnull().loc[(df.isnull()['cast_member_1']==True)&(df.isnull()['genre_1']==True)]

In [4]:
not_found_shows=list(df['show_title'][df_null.index])

In [5]:
not_found_shows

['Shameless (U.S.)',
 'Ginny & Georgia',
 'Sex/Life',
 'New Amsterdam',
 'S.W.A.T.',
 "Kountry Wayne: A Woman's Prayer",
 'Love, Death & Robots',
 'Titans',
 'Harry & Meghan',
 'The Good Doctor',
 'Sex Education',
 'One of Us Is Lying',
 'PAW Patrol',
 'Taboo',
 'Hit & Run',
 'Lockwood & Co.',
 "Grey's Anatomy",
 'Lucifer',
 'Mark Normand: Soup to Nuts',
 'All American',
 'Cocaine Cowboys: The Kings of Miami',
 'Locke & Key',
 'Blood & Water',
 'The Great British Baking Show',
 'Muscles & Mayhem: An Unauthorized Story of American Gladiators',
 'For Life',
 'Married at First Sight']

In [6]:
def get_link_by_title(show_title):
    link='Not Found'
    url = 'http://www.google.com/search?q={} netflix'.format(show_title)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    for i in soup.find_all('a'):
        link_location=str(i).find('https://www.netflix.com/title/')
        if link_location!=-1:
            link=str(i)[link_location::].split('&')[0]
            break
    return link

In [7]:
links=[]
for show in tqdm(not_found_shows):
    links.append(get_link_by_title(show))

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:19<00:00,  1.38it/s]


In [8]:
def get_cast_by_link(url):
    
    r=requests.get(url)
    soup=BeautifulSoup(r.text)
    
    cast=[]
    for i in soup.find_all('span'):
        cast_location=str(i).find('"more-details-item-cast">')
        if cast_location!=-1:
            cast_member=str(i)[cast_location+25::].split('<')[0]
            cast.append(cast_member)
        if len(cast)==3:
            break
            
    while len(cast)!=3:
        cast.append(None)
        
    return cast

In [9]:
def get_genres_by_link(url):
    
    r=requests.get(url)
    soup=BeautifulSoup(r.text)

    genres=[]
    for i in soup.find_all('span'):
        genre_location=str(i).find('"more-details-item-genres">')
        if genre_location!=-1:
            line=str(i)[genre_location::].split('>')
            for j in line:
                try:
                    if j[0].isupper():
                        genres.append(j.split('<')[0])
                except: pass
        if len(genres)==3:
            break
            
    while len(genres)!=3:
        genres.append(None)
    
    return genres

In [10]:
def get_moods_by_link(url):
    
    r=requests.get(url)
    soup=BeautifulSoup(r.text)

    moods=[]
    for i in soup.find_all('span'):
        mood_location=str(i).find('"more-details-item-mood-tag">')
        if mood_location!=-1:
            mood=str(i)[mood_location::].split('>')[1].split('<')[0]
            moods.append(mood)
        if len(moods)==2:
            break
            
    while len(moods)!=2:
        moods.append(None)
        
    return moods

In [11]:
for i in zip(not_found_shows,links):
    print(i)

('Shameless (U.S.)', 'https://www.netflix.com/title/70184207')
('Ginny & Georgia', 'Not Found')
('Sex/Life', 'Not Found')
('New Amsterdam', 'https://www.netflix.com/title/80241181')
('S.W.A.T.', 'https://www.netflix.com/title/80179292')
("Kountry Wayne: A Woman's Prayer", 'https://www.netflix.com/title/81643299')
('Love, Death & Robots', 'Not Found')
('Titans', 'Not Found')
('Harry & Meghan', 'Not Found')
('The Good Doctor', 'Not Found')
('Sex Education', 'Not Found')
('One of Us Is Lying', 'Not Found')
('PAW Patrol', 'Not Found')
('Taboo', 'Not Found')
('Hit & Run', 'Not Found')
('Lockwood & Co.', 'Not Found')
("Grey's Anatomy", 'https://www.netflix.com/title/70140391')
('Lucifer', 'https://www.netflix.com/title/80057918')
('Mark Normand: Soup to Nuts', 'Not Found')
('All American', 'https://www.netflix.com/title/81012998')
('Cocaine Cowboys: The Kings of Miami', 'Not Found')
('Locke & Key', 'Not Found')
('Blood & Water', 'Not Found')
('The Great British Baking Show', 'https://www.net

Some of these shows are not on Netflix anymore, so we are going to cut them from our database.

In [12]:
shows_not_on_netflix=['Shameless (U.S.)','New Amsterdam','S.W.A.T.','The Good Doctor','One of Us Is Lying',\
                       'All American','The Great British Baking Show','For Life','Married at First Sight']

In [13]:
fixed_links=['https://www.netflix.com/title/81025696','https://www.netflix.com/title/80991848',\
          'https://www.netflix.com/title/81643299','https://www.netflix.com/title/80174608',\
          'https://www.netflix.com/title/80218200','https://www.netflix.com/title/81439256',\
          'https://www.netflix.com/title/80197526','https://www.netflix.com/title/80026226',\
          'https://www.netflix.com/title/80158276','https://www.netflix.com/title/80205626',\
          'https://www.netflix.com/title/81116060','https://www.netflix.com/title/70140391',\
          'https://www.netflix.com/title/80057918','https://www.netflix.com/title/81616392',\
          'https://www.netflix.com/title/80996810','https://www.netflix.com/title/80241239',\
          'https://www.netflix.com/title/81044547','https://www.netflix.com/title/81462151']

In [14]:
for show in shows_not_on_netflix:
    not_found_shows.remove(show)

In [15]:
not_found_shows

['Ginny & Georgia',
 'Sex/Life',
 "Kountry Wayne: A Woman's Prayer",
 'Love, Death & Robots',
 'Titans',
 'Harry & Meghan',
 'Sex Education',
 'PAW Patrol',
 'Taboo',
 'Hit & Run',
 'Lockwood & Co.',
 "Grey's Anatomy",
 'Lucifer',
 'Mark Normand: Soup to Nuts',
 'Cocaine Cowboys: The Kings of Miami',
 'Locke & Key',
 'Blood & Water',
 'Muscles & Mayhem: An Unauthorized Story of American Gladiators']

In [16]:
rows=[]

for show,link in tqdm(zip(not_found_shows,fixed_links),total=len(fixed_links)):
    row=[show]
    try:
        url=link
        cast=get_cast_by_link(url)
        genres=get_genres_by_link(url)
        moods=get_moods_by_link(url)
        
        row=row+cast+genres+moods
        rows.append(row)
    except:
        rows.append(row)
        
rows

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [01:49<00:00,  6.06s/it]


[['Ginny & Georgia',
  'Brianne Howey',
  'Antonia Gentry',
  'Diesel La Torraca',
  'Romantic TV Dramas',
  'TV Dramas',
  'Teen TV Shows',
  'Scandalous',
  'Emotional'],
 ['Sex/Life',
  'Sarah Shahi',
  'Mike Vogel',
  'Adam Demos',
  'Romantic TV Dramas',
  'TV Dramas',
  'TV Shows Based on Books',
  'Steamy',
  'Intimate'],
 ["Kountry Wayne: A Woman's Prayer",
  'Kountry Wayne',
  None,
  None,
  'Stand-Up Comedy',
  None,
  None,
  'Raunchy',
  'Witty'],
 ['Love, Death & Robots',
  'Joe Manganiello',
  'Rosario Dawson',
  'Seth Green',
  'Sci-Fi TV',
  'TV Action &amp; Adventure',
  'US TV Shows',
  'Mind-Bending',
  None],
 ['Titans',
  'Brenton Thwaites',
  'Anna Diop',
  'Teagan Croft',
  'Sci-Fi TV',
  'TV Action &amp; Adventure',
  'US TV Shows',
  'Gritty',
  'Dark'],
 ['Harry & Meghan',
  'The Duke and Duchess of Sussex',
  None,
  None,
  'Biographical Documentaries',
  'Docuseries',
  'Social &amp; Cultural Docs',
  'Investigative',
  'Intimate'],
 ['Sex Education',
  'A

In [17]:
fix_for_netflix_db=pd.DataFrame(rows,columns=['show_title','cast_member_1','cast_member_2','cast_member_3',\
                                       'genre_1','genre_2','genre_3','mood_1','mood_2'])

In [19]:
fix_for_netflix_db

,show_title,cast_member_1,cast_member_2,cast_member_3,genre_1,genre_2,genre_3,mood_1,mood_2
0,Ginny & Georgia,Brianne Howey,Antonia Gentry,Diesel La Torraca,Romantic TV Dramas,TV Dramas,Teen TV Shows,Scandalous,Emotional
1,Sex/Life,Sarah Shahi,Mike Vogel,Adam Demos,Romantic TV Dramas,TV Dramas,TV Shows Based on Books,Steamy,Intimate
2,Kountry Wayne: A Woman's Prayer,Kountry Wayne,None,None,Stand-Up Comedy,None,None,Raunchy,Witty
3,"Love, Death & Robots",Joe Manganiello,Rosario Dawson,Seth Green,Sci-Fi TV,TV Action &amp; Adventure,US TV Shows,Mind-Bending,None
4,Titans,Brenton Thwaites,Anna Diop,Teagan Croft,Sci-Fi TV,TV Action &amp; Adventure,US TV Shows,Gritty,Dark
5,Harry & Meghan,The Duke and Duchess of Sussex,None,None,Biographical Documentaries,Docuseries,Social &amp; Cultural Docs,Investigative,Intimate
6,Sex Education,Asa Butterfield,Gillian Anderson,Ncuti Gatwa,TV Dramas,British,Teen TV Shows,Raunchy,Irreverent
7,PAW Patrol,Devan Cohen,Owen Mason,Kallan Holley,Kids' TV,Animal Friends,TV Cartoons,Feel-Good,Exciting
8,Taboo,None,None,None,None,None,None,None,None
9,Hit & Run,Lior Raz,Kaelen Ohm,Sanaa Lathan,TV Dramas,TV Mysteries,TV Action &amp; Adventure,Gritty,Suspenseful


In [20]:
fix_for_netflix_db.to_excel('fix_for_netflix_db.xlsx',index=False)

In [117]:
df=pd.read_excel('netflix_db.xlsx',index_col=None)
fix=pd.read_excel('fix_for_netflix_db.xlsx',index_col=None)

In [118]:
shows_to_fix=fix['show_title'].tolist()
shows_to_fix_index=[df.loc[df['show_title']==show].index for show in fix['show_title'].tolist()]
for idx in shows_to_fix_index:
    df.drop(idx,inplace=True)

In [119]:
df_new=pd.concat([df,fix],ignore_index=True)
len(list(set(df_new.index)))

284

In [123]:
df_null=df_new.isnull().loc[(df_new.isnull()['cast_member_1']==True)&(df_new.isnull()['genre_1']==True)]
df_null.shape

(11, 9)

In [125]:
df_new.drop(df_null.index,inplace=True)
df_new.shape

(273, 9)

In [127]:
df_null=df_new.isnull().loc[(df_new.isnull()['cast_member_1']==True)&(df_new.isnull()['genre_1']==True)]
df_null.shape

(0, 9)

In [126]:
df_new

,show_title,cast_member_1,cast_member_2,cast_member_3,genre_1,genre_2,genre_3,mood_1,mood_2
0,Black Mirror,Jesse Plemons,Cristin Milioti,Jimmi Simpson,TV Dramas,British,Sci-Fi TV,Mind-Bending,Ominous
1,Savage Beauty,Rosemary Zimu,Dumisani Mbebe,Nthati Moshesh,TV Dramas,South African,Social Issue TV Dramas,Emotional,NaN
3,Our Great National Parks,Barack Obama,NaN,NaN,Family Watch Together TV,Nature &amp; Ecology Documentaries,Science &amp; Nature TV,Inspiring,NaN
4,How to Become a Mob Boss,Peter Dinklage,NaN,NaN,Docuseries,US TV Shows,NaN,Dark,NaN
5,Raising Dion,Alisha Wainwright,Ja'Siah Young,Jazmyn Simon,TV Dramas,Family Watch Together TV,TV Shows Based on Comics,Exciting,NaN
...,...,...,...,...,...,...,...,...,...
279,Mark Normand: Soup to Nuts,Mark Normand,NaN,NaN,Stand-Up Comedy,NaN,NaN,Raunchy,Witty
280,Cocaine Cowboys: The Kings of Miami,NaN,NaN,NaN,Courtroom TV Shows,Crime TV Shows,Docuseries,Investigative,NaN
281,Locke & Key,Darby Stanchfield,Connor Jessup,Emilia Jones,TV Dramas,Teen TV Shows,TV Action &amp; Adventure,Mind-Bending,Witty
282,Blood & Water,Ama Qamata,Khosi Ngema,Gail Mabalane,TV Dramas,South African,Teen TV Shows,Suspenseful,NaN


In [132]:
df_new.to_excel('fixed_netflix_db.xlsx',index=False)